MultiRAT preprocessing
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Purpose
In this section I generate the bare bone BIDS folder metadata ahead of an upload to opennenuro.org. Datasets acquired with standardRAT protocol were generated using proper BIDS converter (bidscoin and brkraw), so they contain the right metadata. Other datasets were supplied `as is` by contributing parties. See   [asset preparation](scripts/proj_asset.ipynb) for the detail on how supplied data were converted to NIFTI and BIDS.   

If re-using some of the scripts, please follow citations guidelines for
the software used. I’ve provided the links to the software wherever
possible. See also the [license](../LICENSE.md) for this software.


In [1]:
# init variables
init_folder='/home/traaffneu/joagra/code/MultiRat'  # location of the codes
analysis_folder='/project/4180000.19/multiRat'      # location of the bids folder and results
df_path='../assets/table/multiRat_rest_20220414.tsv'    # meta-data table


In [2]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(df_path, sep='\t')  # load the table

BIDs 1.6.0 requires the following for fMRI. 
1. RepetitionTime
2. TaskName (for func)

The rest is recommended or optional. I am however unsure of the precision of all parameters provided by the users for their respective datasets. For this reason, I only added the following meta-data items to the .json files. 

MagneticFieldStrength
PulseSequenceType
EchoTime

The full table with raw user inputs is provided in the ./assets/table folder. 

In [29]:
import json


for index, row in df.iterrows():
    subject_ID="sub-0"+str(row['rat.sub'])
    session_ID="ses-"+str(row['rat.ses'])

    bidsversion='1.6.0'

    magneticfield=row['MRI.field.strength']

    anatfilename=os.path.join(analysis_folder, 'bids',subject_ID,session_ID,'anat',subject_ID+'_'+session_ID+'_T2w.json')
    anatTR=row['anat.TR']
    anatTE=row['anat.TE']
    anatseq=str(row['anat.sequence'])

    if pd.isna(anatTR):
        anatTR="n/a"
    if pd.isna(anatTE):
        anatTE="n/a"
    

    funcfilename=os.path.join(analysis_folder, 'bids',subject_ID,session_ID,'func',subject_ID+'_'+session_ID+'_task-rest_run-1_bold.json')
    funcTR=row['func.TR']
    funcTE=row['func.TE']
    funcseq=str(row['func.sequence'])

    if pd.isna(funcTR):
        funcTR="n/a"
    if pd.isna(funcTE):
        funcTE="n/a"

    if row['exp.type']=='resting-state':
        taskname='rest'
    # this should only be for the stim evoked dataset.
    else:
        taskname='stim'
        taskonset=str(row['func.sensory.onset'])
        taskduration=str(row['func.sensory.duration'])

    func={
        'MagneticFieldStrength': magneticfield,
        'EchoTime': funcTE,
        'RepetitionTime': funcTR,
        'PulseSequenceType': funcseq}

    anat={
        'MagneticFieldStrength': magneticfield,
        'EchoTime': anatTE,
        'RepetitionTime': anatTR,
        'PulseSequenceType': anatseq}

    with open(anatfilename, 'w') as outfile:
        json.dump(anat, outfile)

    with open(funcfilename, 'w') as outfile:
        json.dump(func, outfile)

In [5]:
import os
import nibabel as nib

for index, row in df.iterrows():

    subject_ID="sub-0"+str(row['rat.sub'])
    session_ID="ses-"+str(row['rat.ses'])

    funcfilename=os.path.join(analysis_folder, 'bids',subject_ID,session_ID,'func',subject_ID+'_'+session_ID+'_task-rest_run-1_bold.nii.gz')
    n1_img = nib.load(funcfilename)
    if n1_img.header['pixdim'][4] != row['func.TR']:
        n1_img.header['pixdim'][4] = row['func.TR']


    n1_img.to_filename(funcfilename)

In [4]:
# ! bids validator in python is moody. seemly working using website (minus some warnings)
from bids_validator import BIDSValidator
BIDSValidator().is_bids(os.path.join(analysis_folder,'bids'))


ModuleNotFoundError: No module named 'bids_validator'